# Fine-tuning do modelo pré-treinado

## Avisos importantes!


*   Como esta etapa demanda muito poder computacional (memória e processamento), faça o upload e execute este notebook no Google Colab, **após** a fase de pré-processamento da base de dados;
*   Logo depois o upload do notebook, verifique o ambiente de execução do Google Colab, clicando na seta para baixo no canto superior direito, e então em "Alterar o tipo de ambiente de execução";
*   Caso a opção selecionada seja 'CPU', troque para 'GPUs: T4' (ou para outra GPU a sua escolha, caso você tenha algum plano pago do Colab);
*   Após isso, faça o upload dos arquivos necessários para o ambiente, clicando no ícone "pasta" (lado esquerdo da tela) e arrastando os seguintes arquivos do repositório:
    * notebooks/requirements_fine_tuning.txt
    * temp/temp_input.parquet

Após essas etapas, inicie a execução deste notebook, e não esqueça de fazer o download o modelo treinado ao final, no diretório especificado.


### Instala e importa os módulos necessários

In [ ]:
!pip install -q -r requirements_fine_tuning.txt

In [ ]:
import sys
import os
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from datasets import Dataset
import joblib

### Carregamento do modelo pré-treinado e dataset


In [ ]:
modelo = SentenceTransformer("sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens")
modelo

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
df = pd.read_parquet("temp_input.parquet")
df

,is_sarcastic,headline,text
0,0,"[agu, pede, arquivamento, ação, pt, anular, de...","[advocacia, geral, união, agu, enviou, manifes..."
1,0,"[compliance, terceirização, contornar, histerese]","[agravamento, cenário, econômico, pandemia, co..."
2,1,"[paulo, barros, novo, técnico, fluminense]","[laranjeiras, minutos, após, demissão, muricy,..."
3,1,"[voto, celso, mello, narrado, galvão, bueno]","[mônaco, após, empate, tempo, regulamentar, cú..."
4,1,"[afastamento, ministro, trabalho, tira, eterna...","[terceirizastok, cristiane, aquece, duas, pala..."
...,...,...,...
6643,0,"[aplicativos, abutres, vivem, cultura, litigio...","[conselho, nacional, justiça, cnj, anuário, di..."
6644,1,"[protesto, contra, raduan, nassar, michel, tem...","[pasárgada, aviltado, discurso, proferido, esc..."
6645,1,"[dilma, captará, apoio, político, via, lei, ro...","[bye, bye, brasil, acossada, impeachment, dilm..."
6646,1,[retrospectiva],"[redação, inspirada, notável, esforço, congres..."


### Preparando dataset para o fine-tuning

In [ ]:
df = df.dropna(subset=['text', 'is_sarcastic'])
df['text'] = df['text'].apply(lambda tokens: " ".join(tokens) if isinstance(tokens, list) else str(tokens))
df['headline'] = df['headline'].apply(lambda tokens: " ".join(tokens) if isinstance(tokens, list) else str(tokens))
df['is_sarcastic'] = df['is_sarcastic'].astype(int)

# Limite opcional de tamanho do texto
df['text'] = df['text'].apply(lambda x: x[:512])

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['is_sarcastic'], random_state=42)




In [ ]:
train_ds = Dataset.from_pandas(train_df.rename(columns={'text': 'text', 'is_sarcastic': 'label'}), preserve_index=False)
train_ds

Dataset({
    features: ['label', 'headline', 'text'],
    num_rows: 5318
})

In [ ]:
eval_ds = Dataset.from_pandas(test_df.rename(columns={'text': 'text', 'is_sarcastic': 'label'}), preserve_index=False)
eval_ds

Dataset({
    features: ['label', 'headline', 'text'],
    num_rows: 1330
})

### Define parâmetros de treinamento

In [ ]:
def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        preds = predictions.argmax(axis=1)
        return {"accuracy": accuracy_score(labels, preds)}

training_args = SentenceTransformerTrainingArguments(
    output_dir="modelo_finetunado_sarcasmo",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    logging_steps=10,
    save_total_limit=1,
    learning_rate=2e-5,
    warmup_steps=10,
    fp16=False,
    report_to="none" # Adiciona esta linha para desabilitar o wandb
)

train_loss = losses.SoftmaxLoss(
    model=modelo,
    sentence_embedding_dimension=modelo.get_sentence_embedding_dimension(),
    num_labels=2
)

trainer = SentenceTransformerTrainer(
    model=modelo,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss=train_loss,
    compute_metrics=compute_metrics
)

### Realiza o treinamento

In [ ]:
trainer.train()

Step,Training Loss
10,0.652400
20,0.516400
30,0.520100
40,0.560400
50,0.353500
60,0.320900
70,0.345800
80,0.353300
90,0.332300
100,0.262300


TrainOutput(global_step=1332, training_loss=0.12662978195088884, metrics={'train_runtime': 1209.2136, 'train_samples_per_second': 17.592, 'train_steps_per_second': 1.102, 'total_flos': 0.0, 'train_loss': 0.12662978195088884, 'epoch': 4.0})

### Treina classificador

In [ ]:
# Gerando embeddings para os conjuntos de treino e teste
X_train = modelo.encode(train_df['text'].tolist(), convert_to_tensor=True).cpu().numpy()
X_test = modelo.encode(test_df['text'].tolist(), convert_to_tensor=True).cpu().numpy()
y_train = train_df['is_sarcastic'].values
y_test = test_df['is_sarcastic'].values

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precisão (weighted): {precision_score(y_test, y_pred, average='weighted'):.4f}")
print(f"Recall (weighted): {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_pred, average='weighted'):.4f}")


Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       443
           1       0.94      0.95      0.95       887

    accuracy                           0.93      1330
   macro avg       0.92      0.92      0.92      1330
weighted avg       0.93      0.93      0.93      1330

Acurácia: 0.9293
Precisão (weighted): 0.9290
Recall (weighted): 0.9293
F1-Score (weighted): 0.9291


### Salva modelo treinado e classificador

In [ ]:
modelo.save("modelo_finetunado_sarcasmo")
joblib.dump(clf, os.path.join("modelo_finetunado_sarcasmo/classificador_logreg.pkl"))

['modelo_finetunado_sarcasmo/classificador_logreg.pkl']

Faz o download da pasta, demora alguns minutos

In [ ]:
from google.colab import files
import shutil
import os

folder_to_download = "modelo_finetunado_sarcasmo"
zip_filename = f"{folder_to_download}.zip"

# Compacta a pasta, criando o zip a partir do diretório pai
shutil.make_archive(folder_to_download, 'zip', root_dir='.', base_dir=folder_to_download)

# Baixa o arquivo zipado
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Após o download, descompacte o arquivo .zip na pasta 'modelos' do repositório
